# **Face Detection using Haar Cascade**

In [2]:
import cv2
photo = cv2.VideoCapture(0)
faceClassifer = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
while True:
    status, img = photo.read()
    faceCoord = faceClassifer.detectMultiScale(img)

    if len(faceCoord) == 1:
        x = faceCoord[0][0]
        y = faceCoord[0][1]
        w = faceCoord[0][2]
        h = faceCoord[0][3]

        #cv2.rectangle(img, (x, y), (x+w, y+h), [0, 255, 0], 1)
        img = img[y:y+h, x:x+w]

    cv2.imshow("Face", img)

    if cv2.waitKey(1) & 0xFF == ord('c'):
        break


photo.release()
cv2.destroyAllWindows()


# **Bulrring The Face using Haar Cascade**

In [3]:
import cv2
photo = cv2.VideoCapture(0)
faceClassifer = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def blurImage(img, faces):
    result_image = img.copy()
    #result_image = cv2.GaussianBlur(result_image, (21, 21), 0)
    for(x, y, w, h) in faces:
        face = result_image[y:y+h, x:x+w]
        face = cv2.GaussianBlur(face, (0, 0), 30)
        result_image[y:y+h, x:x+w] = face
    return result_image

while True:
    status, img = photo.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faceCoord = faceClassifer.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    blurred = blurImage(img, faceCoord)

    cv2.imshow('Blur', blurred)
    if cv2.waitKey(1) & 0xFF == ord('c'):
        break

photo.release()
cv2.destroyAllWindows()

# **Detecting Faces along with the Distance from Camera**

In [6]:
import cv2
import math

photo = cv2.VideoCapture(0)
faceClassifer = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
KNOWN_FACE_WIDTH = 15
FOCAL_LENGTH = 600

def findDistance(face_width):
    dist = (KNOWN_FACE_WIDTH * FOCAL_LENGTH) / face_width
    return dist

while True:
    status, img = photo.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faceCoord = faceClassifer.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for(x, y, w, h) in faceCoord:
        dist = findDistance(w)
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(img, f"Distance: {dist:.2f} cm", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 1)
    
    cv2.imshow('Face Distance', img)
    if cv2.waitKey(1) & 0xFF == ord('c'):
        break

photo.release()
cv2.destroyAllWindows()

# **Adding Accessories**

In [7]:
import cv2
import numpy as np

photo = cv2.VideoCapture(0)
faceClassifer = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eyeClassifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
glasses_img = cv2.imread("glasses.png", cv2.IMREAD_UNCHANGED)

def placeGlasses(image, x, y, w, h):

    roi = image[y:y+h, x:x+w]
    eyes = eyeClassifier.detectMultiScale(roi)

    if len(eyes) >= 2:
        eyes = sorted(eyes, key=lambda x: x[0])
        e1_x, e1_y, e1_w, e1_h = eyes[0]
        e2_x, e2_y, e2_w, e2_h = eyes[1]
        e1_c = (x + e1_x + e1_w // 2, y + e1_y + e1_h // 2)
        e2_c = (x + e2_x + e2_w // 2, y + e2_y + e2_h // 2)

        eyes_distance = e2_c[0] - e1_c[0]
        glasses_width = int(1.5 * eyes_distance)
        glasses_height = int(glasses_width * glasses_img.shape[0] / glasses_img.shape[1])

        glasses_x = e1_c[0] - int(0.25 * eyes_distance)
        glasses_y = e1_c[1] - int(0.4 * glasses_height)

        glasses_resized = cv2.resize(glasses_img, (glasses_width, glasses_height))

        mask = glasses_resized[:, :, 3]
        mask_inv = cv2.bitwise_not(mask)

        glasses_colored = glasses_resized[:, :, 0:3]

        roi = img[glasses_y:glasses_y + glasses_height, glasses_x:glasses_x + glasses_width]
        roi_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
        roi_fg = cv2.bitwise_and(glasses_colored, glasses_colored, mask=mask)

        dst = cv2.add(roi_bg, roi_fg)
        img[glasses_y:glasses_y + glasses_height, glasses_x:glasses_x + glasses_width] =  dst



while True:
    status, img = photo.read()
    if not status:
        break
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = faceClassifer.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for(x, y, w, h) in faces:
        placeGlasses(img, x, y, w, h)

    cv2.imshow("Glasses", img)
    if cv2.waitKey(1) & 0xFF == ord('c'):
        break

photo.release()
cv2.destroyAllWindows()

       

